In [6]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone
import sqlalchemy

# Making the DataFrame

## Creating a function for multiple airports

In [10]:
icao_list = ["EDDB"]

flights_to_db = get_flight_data(icao_list)
flights_to_db

,arrival_airport_icao,departure_airport_icao,departure_airport_name,scheduled_arrival_time,flight_number,data_retrieved_at
0,EDDB,ZBAA,Beijing,2024-06-14 06:45:00,HU 489,2024-06-13 14:18:13
1,EDDB,KEWR,Newark,2024-06-14 07:15:00,UA 962,2024-06-13 14:18:13
2,EDDB,OTHH,Doha,2024-06-14 07:20:00,QR 79,2024-06-13 14:18:13
3,EDDB,EDDS,Stuttgart,2024-06-14 07:25:00,EW 2002,2024-06-13 14:18:13
4,EDDB,EDDF,Frankfurt-am-Main,2024-06-14 07:25:00,LH 172,2024-06-13 14:18:13
...,...,...,...,...,...,...
341,EDDB,LEMD,Madrid,2024-06-14 23:00:00,I2 3674,2024-06-13 14:18:14
342,EDDB,LEMD,Madrid,2024-06-14 23:00:00,IB 3674,2024-06-13 14:18:14
343,EDDB,LFPO,Paris,2024-06-14 23:10:00,TO 3412,2024-06-13 14:18:14
344,EDDB,LGKO,Kos Island,2024-06-14 23:15:00,A3 3966,2024-06-13 14:18:14


Your flight function can now be incorporated with your other functions to send and receive data from your SQL database.

In [ ]:
icao_list = ["EDDB"]

flights_to_db = get_flight_data(icao_list)

In [8]:
flights_to_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   arrival_airport_icao    261 non-null    object        
 1   departure_airport_icao  261 non-null    object        
 2   departure_airport_name  261 non-null    object        
 3   scheduled_arrival_time  261 non-null    datetime64[ns]
 4   flight_number           261 non-null    object        
 5   data_retrieved_at       261 non-null    datetime64[ns]
dtypes: datetime64[ns](2), object(4)
memory usage: 12.4+ KB


# Create "flight" table in the database

```sql
CREATE TABLE flights(
	flight_id INT AUTO_INCREMENT,
    arrival_airport_icao VARCHAR(10),
    departure_airport_icao VARCHAR(10),
    departure_airport_name VARCHAR(30),
    scheduled_arrival_time DATETIME,
    flight_number VARCHAR(30),
    data_retrieved_at DATETIME,
    PRIMARY KEY (flight_id),
    FOREIGN KEY (arrival_airport_icao) REFERENCES airports(icao)
);
```

# Push the "flights_to_db" to the empty "flights" table in the database

In [14]:
# from Keys import MySQL_pass

schema = "gans_local" # The name of your database
host = "127.0.0.1"
user = "root"
password = "PedregalNuria7178"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [16]:
flights_to_db.to_sql('flights',
                    if_exists='append',
                    con=connection_string,
                    index=False)

346

In [17]:
pd.read_sql("flights", con=connection_string)

,flight_id,arrival_airport_icao,departure_airport_icao,departure_airport_name,scheduled_arrival_time,flight_number,data_retrieved_at
0,1,EDDB,LTBJ,İzmir,2024-06-11 06:30:00,XQ 966,2024-06-10 09:40:30
1,2,EDDB,KEWR,Newark,2024-06-11 07:15:00,UA 962,2024-06-10 09:40:30
2,3,EDDB,OTHH,Doha,2024-06-11 07:20:00,QR 79,2024-06-10 09:40:30
3,4,EDDB,EDDS,Stuttgart,2024-06-11 07:25:00,EW 2002,2024-06-10 09:40:30
4,5,EDDB,EDDF,Frankfurt-am-Main,2024-06-11 07:25:00,LH 172,2024-06-10 09:40:30
...,...,...,...,...,...,...,...
256,257,EDDB,EGCC,Manchester,2024-06-11 23:00:00,FR 1155,2024-06-10 09:40:31
257,258,EDDB,LEMD,Madrid,2024-06-11 23:00:00,IB 3674,2024-06-10 09:40:31
258,259,EDDB,LEMD,Madrid,2024-06-11 23:10:00,IB 1809,2024-06-10 09:40:31
259,260,EDDB,LFPO,Paris,2024-06-11 23:10:00,TO 3412,2024-06-10 09:40:31


# Master function

In [18]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone
import sqlalchemy

def retreiving_and_sending_data_flights():
  connection_string = connection()
  airports_df = get_airports_data(connection_string)
  flights_df = get_flights_data(airports_df)
  send_flights_data(flights_df, connection_string)
  return "Data has been updated"

def connection():
  schema = "gans_local" # The name of your database
  host = "127.0.0.1"
  user = "root"
  password = "PedregalNuria7178"
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'


def get_airports_data(connection_string):
  return pd.read_sql("airports", con=connection_string)


def get_flights_data(airports_df):
  api_key = "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4" # USE THE PROVIDED KEY

  berlin_timezone = timezone('Europe/Berlin')
  today = datetime.now(berlin_timezone).date()
  tomorrow = (today + timedelta(days=1))

  flight_items = []



  for icao in airports_df["icao"]:
    # the api can only make 12 hour calls, therefore, 2 12 hour calls make a full day
    # using the nested lists below we can make a morning call and extract the data
    # then make an afternoon call and extract the data
    times = [["00:00","11:59"],["12:00","23:59"]]

    for time in times:
      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

      querystring = {"withLeg":"true",
                    "direction":"Arrival",
                    "withCancelled":"false",
                    "withCodeshared":"true",
                    "withCargo":"false",
                    "withPrivate":"false"}

      headers = {
          'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
          'x-rapidapi-key': api_key
          }

      response = requests.get(url, headers=headers, params=querystring)

      flights_json = response.json()


      retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
      icao_id = airports_df.loc[airports_df["icao"] == icao, "icao"].values[0]

      for item in flights_json["arrivals"]:
        flight_item = {
            "arrival_airport_icao": icao_id,
            "departure_airport_icao": item["departure"]["airport"].get("icao", None),
            "departure_airport_name": item["departure"]["airport"].get("name", None),
            "scheduled_arrival_time": item["arrival"]["scheduledTime"].get("local", None),
            "flight_number": item.get("number", None),
            "data_retrieved_at": retrieval_time
        }

        flight_items.append(flight_item)

  flights_df = pd.DataFrame(flight_items)
  flights_df["scheduled_arrival_time"] = flights_df["scheduled_arrival_time"].str[:-6]
  flights_df["scheduled_arrival_time"] = pd.to_datetime(flights_df["scheduled_arrival_time"])
  flights_df["data_retrieved_at"] = pd.to_datetime(flights_df["data_retrieved_at"])

  return flights_df

def send_flights_data(flights_df, connection_string):
  flights_df.to_sql('flights',
                    if_exists='append',
                    con=connection_string,
                    index=False)


In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone
import sqlalchemy

@functions_framework.http
def flight_update(request):
    schema = "gans_cloud" # The name of your database
    host = "35.205.77.170"
    user = "root"
    password = "PedregalNuria7178"
    port = 3306

   connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'


def get_airports_data(connection_string):
  return pd.read_sql("airports", con=connection_string)


def get_flights_data(airports_df):
  api_key = "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4" # USE THE PROVIDED KEY

  berlin_timezone = timezone('Europe/Berlin')
  today = datetime.now(berlin_timezone).date()
  tomorrow = (today + timedelta(days=1))

  flight_items = []



  for icao in airports_df["icao"]:
    # the api can only make 12 hour calls, therefore, 2 12 hour calls make a full day
    # using the nested lists below we can make a morning call and extract the data
    # then make an afternoon call and extract the data
    times = [["00:00","11:59"],["12:00","23:59"]]

    for time in times:
      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

      querystring = {"withLeg":"true",
                    "direction":"Arrival",
                    "withCancelled":"false",
                    "withCodeshared":"true",
                    "withCargo":"false",
                    "withPrivate":"false"}

      headers = {
          'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
          'x-rapidapi-key': api_key
          }

      response = requests.get(url, headers=headers, params=querystring)

      flights_json = response.json()


      retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
      icao_id = airports_df.loc[airports_df["icao"] == icao, "icao"].values[0]

      for item in flights_json["arrivals"]:
        flight_item = {
            "arrival_airport_icao": icao_id,
            "departure_airport_icao": item["departure"]["airport"].get("icao", None),
            "departure_airport_name": item["departure"]["airport"].get("name", None),
            "scheduled_arrival_time": item["arrival"]["scheduledTime"].get("local", None),
            "flight_number": item.get("number", None),
            "data_retrieved_at": retrieval_time
        }

        flight_items.append(flight_item)

  flights_df = pd.DataFrame(flight_items)
  flights_df["scheduled_arrival_time"] = flights_df["scheduled_arrival_time"].str[:-6]
  flights_df["scheduled_arrival_time"] = pd.to_datetime(flights_df["scheduled_arrival_time"])
  flights_df["data_retrieved_at"] = pd.to_datetime(flights_df["data_retrieved_at"])

  return flights_df

def send_flights_data(flights_df, connection_string):
  flights_df.to_sql('flights',
                    if_exists='append',
                    con=connection_string,
                    index=False)